## Installing and Importing Packages

In [2]:
%pip install nltk

Note: you may need to restart the kernel to use updated packages.


In [3]:
import nltk
nltk.download()

showing info https://raw.githubusercontent.com/nltk/nltk_data/gh-pages/index.xml


True

In [37]:
import pandas as pd
pd.set_option('display.max_colwidth', 100)
import numpy as np
import re
import string 

stopwords = nltk.corpus.stopwords.words('english')
ps = nltk.PorterStemmer()

from sklearn.model_selection import train_test_split , GridSearchCV
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.ensemble import RandomForestClassifier , GradientBoostingClassifier

In [5]:
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)

# Reading the Data

In [6]:
imdb_prelim = pd.read_excel("IMDB_dataset.xlsx")
imdb_prelim.head()

,review,sentiment
0,"I thought this was a wonderful way to spend time on a too hot summer weekend, sitting in the air...",positive
1,"Probably my all-time favorite movie, a story of selflessness, sacrifice and dedication to a nobl...",positive
2,I sure would like to see a resurrection of a up dated Seahunt series with the tech they have tod...,positive
3,"This show was an amazing, fresh & innovative idea in the 70's when it first aired. The first 7 o...",negative
4,Encouraged by the positive comments about this film on here I was looking forward to watching th...,negative


In [7]:
imdb_prelim['review'].loc[23]

"It had all the clichÃ©s of movies of this type and no substance. The plot went nowhere and at the end of the movie I felt like a sucker for watching it. The production was good; however, the script and acting were B-movie quality. The casting was poor because there were good actors mixed in with crumby actors. The good actors didn't hold their own nor did they lift up the others. <br /><br />This movie is not worthy of more words, but I will say more to meet the minimum requirement of ten lines. James Wood and Cuba Gooding, Jr. play caricatures of themselves in other movies. <br /><br />If you are looking for mindless entertainment, I still wouldn't recommend this movie."

In [8]:
imdb_prelim.describe()

,review,sentiment
count,25000,25000
unique,24898,2
top,"When i got this movie free from my job, along with three other similar movies.. I watched then w...",positive
freq,3,12500


In [9]:
imdb_prelim.sentiment.value_counts()

positive    12500
negative    12500
Name: sentiment, dtype: int64

#### We can see that the dataset is balanced.

## Data Preprocessing

### Converting 'reviews' to Lowercase

In [10]:
imdb_prelim['review'] = imdb_prelim['review'].str.lower()
imdb_prelim.head()

,review,sentiment
0,"i thought this was a wonderful way to spend time on a too hot summer weekend, sitting in the air...",positive
1,"probably my all-time favorite movie, a story of selflessness, sacrifice and dedication to a nobl...",positive
2,i sure would like to see a resurrection of a up dated seahunt series with the tech they have tod...,positive
3,"this show was an amazing, fresh & innovative idea in the 70's when it first aired. the first 7 o...",negative
4,encouraged by the positive comments about this film on here i was looking forward to watching th...,negative


### Removing HTML tags

In [11]:
def remove_html_tags (text):
    rmv = re.compile('<.*?>')
    return rmv.sub(r'', text)

In [12]:
imdb_prelim['review'] = imdb_prelim['review'].apply(remove_html_tags)
imdb_prelim['review'].loc[:24]

0     i thought this was a wonderful way to spend time on a too hot summer weekend, sitting in the air...
1     probably my all-time favorite movie, a story of selflessness, sacrifice and dedication to a nobl...
2     i sure would like to see a resurrection of a up dated seahunt series with the tech they have tod...
3     this show was an amazing, fresh & innovative idea in the 70's when it first aired. the first 7 o...
4     encouraged by the positive comments about this film on here i was looking forward to watching th...
5     phil the alien is one of those quirky films where the humour is based around the oddness of ever...
6     i saw this movie when i was about 12 when it came out. i recall the scariest scene was the big b...
7     so im not a big fan of boll's work but then again not many are. i enjoyed his movie postal (mayb...
8     this a fantastic movie of three prisoners who become famous. one of the actors is george clooney...
9     this movie made it into one of my top 10

### Removing URLs

In [13]:
def remove_url(text):
    re_url = re.compile('https?://\S+|www\.\S+')
    return re_url.sub('', text)

In [14]:
imdb_prelim['review'] = imdb_prelim['review'].apply(remove_url)
imdb_prelim['review'].loc[2336]

"a message movie, but a rather good one. outstanding cast, top to bottom. interesting in that bette davis's plot line is essentially back story! the extremely negative reviews (name throwing at the screenplay/playwright, associating this somehow with extremely negative comments about 'angles in america', etc. etc.) object to the movie being too preachy about germany in wwii. gosh, that is just a bit too sophisticated an understanding of morality for me.theatrical and movie-making, and acting styles vary over time and of course 70 years later this particular movie would not be made in this way. yes casablanca is a better movie (i guess), but although made in the same year and both having nazis in them, casablanca is primarily a love story. the love story in this movie takes second seat to the spy plot--more of a thriller. both have a rather large number of somewhat cheesy accents and wonderful character actors. the children are a bit tedious and could have been edited"

## Removing Punctuations

In [15]:
def remove_punct(text):
    re_punct = "".join([char for char in text if char not in string.punctuation])
    return re_punct

In [16]:
imdb_prelim['review'] = imdb_prelim['review'].apply(remove_punct)
imdb_prelim.head()

,review,sentiment
0,i thought this was a wonderful way to spend time on a too hot summer weekend sitting in the air ...,positive
1,probably my alltime favorite movie a story of selflessness sacrifice and dedication to a noble c...,positive
2,i sure would like to see a resurrection of a up dated seahunt series with the tech they have tod...,positive
3,this show was an amazing fresh innovative idea in the 70s when it first aired the first 7 or 8 ...,negative
4,encouraged by the positive comments about this film on here i was looking forward to watching th...,negative


## Tokenization 

In [17]:
def tokenize(text):
    tokens = re.split('\W+', text)
    return tokens

In [18]:
imdb_prelim['review_token'] = imdb_prelim['review'].apply(tokenize)
imdb_prelim.head()

,review,sentiment,review_token
0,i thought this was a wonderful way to spend time on a too hot summer weekend sitting in the air ...,positive,"[i, thought, this, was, a, wonderful, way, to, spend, time, on, a, too, hot, summer, weekend, si..."
1,probably my alltime favorite movie a story of selflessness sacrifice and dedication to a noble c...,positive,"[probably, my, alltime, favorite, movie, a, story, of, selflessness, sacrifice, and, dedication,..."
2,i sure would like to see a resurrection of a up dated seahunt series with the tech they have tod...,positive,"[i, sure, would, like, to, see, a, resurrection, of, a, up, dated, seahunt, series, with, the, t..."
3,this show was an amazing fresh innovative idea in the 70s when it first aired the first 7 or 8 ...,negative,"[this, show, was, an, amazing, fresh, innovative, idea, in, the, 70s, when, it, first, aired, th..."
4,encouraged by the positive comments about this film on here i was looking forward to watching th...,negative,"[encouraged, by, the, positive, comments, about, this, film, on, here, i, was, looking, forward,..."


## Removing Stopwords

In [19]:
from nltk.corpus import stopwords
stopwords_english = stopwords.words('english')

In [20]:
def remove_stopwords(text):
    re_stp = [word for word in text if word not in stopwords_english]
    return re_stp

In [21]:
imdb_prelim['review_stopwords'] = imdb_prelim['review_token'].apply(remove_stopwords)
imdb_prelim.head()

,review,sentiment,review_token,review_stopwords
0,i thought this was a wonderful way to spend time on a too hot summer weekend sitting in the air ...,positive,"[i, thought, this, was, a, wonderful, way, to, spend, time, on, a, too, hot, summer, weekend, si...","[thought, wonderful, way, spend, time, hot, summer, weekend, sitting, air, conditioned, theater,..."
1,probably my alltime favorite movie a story of selflessness sacrifice and dedication to a noble c...,positive,"[probably, my, alltime, favorite, movie, a, story, of, selflessness, sacrifice, and, dedication,...","[probably, alltime, favorite, movie, story, selflessness, sacrifice, dedication, noble, cause, p..."
2,i sure would like to see a resurrection of a up dated seahunt series with the tech they have tod...,positive,"[i, sure, would, like, to, see, a, resurrection, of, a, up, dated, seahunt, series, with, the, t...","[sure, would, like, see, resurrection, dated, seahunt, series, tech, today, would, bring, back, ..."
3,this show was an amazing fresh innovative idea in the 70s when it first aired the first 7 or 8 ...,negative,"[this, show, was, an, amazing, fresh, innovative, idea, in, the, 70s, when, it, first, aired, th...","[show, amazing, fresh, innovative, idea, 70s, first, aired, first, 7, 8, years, brilliant, thing..."
4,encouraged by the positive comments about this film on here i was looking forward to watching th...,negative,"[encouraged, by, the, positive, comments, about, this, film, on, here, i, was, looking, forward,...","[encouraged, positive, comments, film, looking, forward, watching, film, bad, mistake, ive, seen..."


## Stemming 

In [22]:
from nltk.stem.porter import PorterStemmer
ps = PorterStemmer()
def perform_stemming(text):
    new_text = [ps.stem(word) for word in text]
    return ' '.join(new_text)

In [23]:
imdb_prelim['review_stemmed'] = imdb_prelim['review_stopwords'].apply(perform_stemming)
imdb_prelim.head()

,review,sentiment,review_token,review_stopwords,review_stemmed
0,i thought this was a wonderful way to spend time on a too hot summer weekend sitting in the air ...,positive,"[i, thought, this, was, a, wonderful, way, to, spend, time, on, a, too, hot, summer, weekend, si...","[thought, wonderful, way, spend, time, hot, summer, weekend, sitting, air, conditioned, theater,...",thought wonder way spend time hot summer weekend sit air condit theater watch lightheart comedi ...
1,probably my alltime favorite movie a story of selflessness sacrifice and dedication to a noble c...,positive,"[probably, my, alltime, favorite, movie, a, story, of, selflessness, sacrifice, and, dedication,...","[probably, alltime, favorite, movie, story, selflessness, sacrifice, dedication, noble, cause, p...",probabl alltim favorit movi stori selfless sacrific dedic nobl caus preachi bore never get old d...
2,i sure would like to see a resurrection of a up dated seahunt series with the tech they have tod...,positive,"[i, sure, would, like, to, see, a, resurrection, of, a, up, dated, seahunt, series, with, the, t...","[sure, would, like, see, resurrection, dated, seahunt, series, tech, today, would, bring, back, ...",sure would like see resurrect date seahunt seri tech today would bring back kid excit mei grew b...
3,this show was an amazing fresh innovative idea in the 70s when it first aired the first 7 or 8 ...,negative,"[this, show, was, an, amazing, fresh, innovative, idea, in, the, 70s, when, it, first, aired, th...","[show, amazing, fresh, innovative, idea, 70s, first, aired, first, 7, 8, years, brilliant, thing...",show amaz fresh innov idea 70 first air first 7 8 year brilliant thing drop 1990 show realli fun...
4,encouraged by the positive comments about this film on here i was looking forward to watching th...,negative,"[encouraged, by, the, positive, comments, about, this, film, on, here, i, was, looking, forward,...","[encouraged, positive, comments, film, looking, forward, watching, film, bad, mistake, ive, seen...",encourag posit comment film look forward watch film bad mistak ive seen 950 film truli one worst...


## TF-IDF Vectorization

### Read and Clean the dataset

In [24]:
imdb = pd.read_excel("IMDB_dataset.xlsx")
imdb.columns = ['review', 'sentiment']
imdb.head()

,review,sentiment
0,"I thought this was a wonderful way to spend time on a too hot summer weekend, sitting in the air...",positive
1,"Probably my all-time favorite movie, a story of selflessness, sacrifice and dedication to a nobl...",positive
2,I sure would like to see a resurrection of a up dated Seahunt series with the tech they have tod...,positive
3,"This show was an amazing, fresh & innovative idea in the 70's when it first aired. The first 7 o...",negative
4,Encouraged by the positive comments about this film on here I was looking forward to watching th...,negative


### Create feature for text message length and % of text that is punctuation

In [25]:
def count_punct(text):
    count = sum([1 for char in text if char in string.punctuation])
    return round(count/(len(text) - text.count(" ")), 3)

imdb['body_len'] = imdb['review'].apply(lambda x: len(x) - x.count(" "))
imdb['punct%'] = imdb['review'].apply(lambda x: count_punct(x))

imdb.head()

,review,sentiment,body_len,punct%
0,"I thought this was a wonderful way to spend time on a too hot summer weekend, sitting in the air...",positive,761,0.053
1,"Probably my all-time favorite movie, a story of selflessness, sacrifice and dedication to a nobl...",positive,538,0.052
2,I sure would like to see a resurrection of a up dated Seahunt series with the tech they have tod...,positive,577,0.021
3,"This show was an amazing, fresh & innovative idea in the 70's when it first aired. The first 7 o...",negative,761,0.043
4,Encouraged by the positive comments about this film on here I was looking forward to watching th...,negative,552,0.056


### Cleaning Punctuations and performing Tokenization 

In [26]:
def clean_text(text):
    text = "".join([word.lower() for word in text if word not in string.punctuation])
    tokens = re.split('\W+', text)
    text = [ps.stem(word) for word in tokens if word not in stopwords]
    return text

### Apply TfidfVectorizer

In [27]:
stopwords = nltk.corpus.stopwords.words('english')
ps = nltk.PorterStemmer()

tfidf_vect_sample = TfidfVectorizer(analyzer=clean_text, max_features=5000)
X_tfidf_sample = tfidf_vect_sample.fit_transform(imdb['review'])
print(X_tfidf_sample.shape)
print(tfidf_vect_sample.get_feature_names_out())

(25000, 5000)
['' '0' '1' ... 'zoom' 'â' 'ã']


In [28]:
X_features = pd.concat([imdb['body_len'], imdb['punct%'], pd.DataFrame(X_tfidf_sample.toarray())], axis=1)
X_features.head()

,body_len,punct%,0,1,2,3,4,5,6,7,...,4990,4991,4992,4993,4994,4995,4996,4997,4998,4999
0,761,0.053,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,538,0.052,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,577,0.021,0.0,0.0,0.0,0.108878,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,761,0.043,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,552,0.056,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


### Vectorizers output sparse matrices

In [29]:
X_tfidf_df = pd.DataFrame(X_tfidf_sample.toarray())
X_tfidf_df.columns = tfidf_vect_sample.get_feature_names_out()
X_tfidf_df

,,0,1,10,100,1000,1010,10br,11,110,...,your,youth,youtub,youv,zero,zombi,zone,zoom,â,ã
0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.108878,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24995,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
24996,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
24997,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
24998,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


## Using GridSearchCV on Random Forest

In [30]:
rf = RandomForestClassifier()
param = {'n_estimators': [10, 25, 50, 75],
        'max_depth': [30, 60, 90, None]}

gs = GridSearchCV(rf, param, cv=5, n_jobs=-1)
gs_fit = gs.fit(X_tfidf_df, imdb['sentiment'])
pd.DataFrame(gs_fit.cv_results_).sort_values('mean_test_score', ascending=False)[0:5]

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_max_depth,param_n_estimators,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
7,190.626401,7.195601,2.252400,0.406578,60,75,"{'max_depth': 60, 'n_estimators': 75}",0.8384,0.8518,0.8410,0.8420,0.8414,0.84292,0.004607,1
15,136.205416,6.970463,0.582383,0.156314,None,75,"{'max_depth': None, 'n_estimators': 75}",0.8328,0.8566,0.8410,0.8450,0.8370,0.84248,0.008144,2
11,199.466200,3.768933,5.223601,2.721443,90,75,"{'max_depth': 90, 'n_estimators': 75}",0.8380,0.8510,0.8374,0.8408,0.8446,0.84236,0.005014,3
3,140.238799,5.972223,4.244600,3.047519,30,75,"{'max_depth': 30, 'n_estimators': 75}",0.8358,0.8456,0.8336,0.8388,0.8396,0.83868,0.004071,4
6,127.580800,4.249169,2.017400,0.485326,60,50,"{'max_depth': 60, 'n_estimators': 50}",0.8308,0.8470,0.8362,0.8342,0.8394,0.83752,0.005501,5


## Using GridSearchCV on Gradient Boosting Classifier

In [36]:
gb = GradientBoostingClassifier()
param = {
    'n_estimators': [1,3,5,7],
    'max_depth': [5,7,9],
    'learning_rate': [0.1]
}

gs2 = GridSearchCV(gb, param, cv=5, n_jobs=-1)
gs2_fit = gs2.fit(X_tfidf_df, imdb['sentiment'])
pd.DataFrame(gs2_fit.cv_results_).sort_values('mean_test_score', ascending=False)[0:5]

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_learning_rate,param_max_depth,param_n_estimators,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
11,395.187996,10.132773,0.223406,0.086645,0.1,9,7,"{'learning_rate': 0.1, 'max_depth': 9, 'n_estimators': 7}",0.7500,0.7574,0.7438,0.7600,0.7514,0.75252,0.005717,1
10,361.163016,33.930021,0.504998,0.143876,0.1,9,5,"{'learning_rate': 0.1, 'max_depth': 9, 'n_estimators': 5}",0.7488,0.7516,0.7410,0.7508,0.7454,0.74752,0.003900,2
7,508.958348,13.756940,2.760804,3.090839,0.1,7,7,"{'learning_rate': 0.1, 'max_depth': 7, 'n_estimators': 7}",0.7302,0.7546,0.7388,0.7420,0.7464,0.74240,0.008085,3
9,263.642550,3.963824,1.730794,1.591923,0.1,9,3,"{'learning_rate': 0.1, 'max_depth': 9, 'n_estimators': 3}",0.7346,0.7432,0.7374,0.7474,0.7436,0.74124,0.004609,4
6,357.244804,8.617922,1.143995,0.086308,0.1,7,5,"{'learning_rate': 0.1, 'max_depth': 7, 'n_estimators': 5}",0.7272,0.7408,0.7306,0.7274,0.7284,0.73088,0.005105,5


In [47]:
# Best parameters for Random Forest
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score, precision_score, recall_score


best_rf_params = gs_fit.best_params_

# Create and fit the Random Forest model with the best parameters
best_rf_model = RandomForestClassifier(**best_rf_params)
best_rf_model.fit(X_tfidf_df, imdb['sentiment'])

# Evaluate the Random Forest model
rf_predictions = best_rf_model.predict(X_tfidf_df)  
rf_accuracy = accuracy_score(imdb['sentiment'], rf_predictions)
rf_precision = precision_score(imdb['sentiment'], rf_predictions, pos_label='positive')
rf_recall = recall_score(imdb['sentiment'], rf_predictions, pos_label='positive')
rf_f1 = f1_score(imdb['sentiment'], rf_predictions, pos_label='positive')

print("Random Forest Evaluation Metrics:")
print(f"Best Parameters: {best_rf_params}")
print(f"Accuracy: {rf_accuracy}")
print(f"Precision: {rf_precision}")
print(f"Recall: {rf_recall}")
print(f"F1 Score: {rf_f1}")

Random Forest Evaluation Metrics:
Best Parameters: {'max_depth': 60, 'n_estimators': 75}
Accuracy: 0.9974
Precision: 0.9948269001193792
Recall: 1.0
F1 Score: 0.997406742469579


In [45]:
# Best parameters for Gradient Boosting
best_gb_params = gs2_fit.best_params_

# Create and fit the Gradient Boosting model with the best parameters
best_gb_model = GradientBoostingClassifier(**best_gb_params)
best_gb_model.fit(X_tfidf_df, imdb['sentiment'])

# Evaluate the Gradient Boosting model
gb_predictions = best_gb_model.predict(X_tfidf_df)  # You might want to use a separate test set for evaluation
gb_accuracy = accuracy_score(imdb['sentiment'], gb_predictions)
gb_precision = precision_score(imdb['sentiment'], gb_predictions, pos_label='positive')
gb_recall = recall_score(imdb['sentiment'], gb_predictions, pos_label='positive')
gb_f1 = f1_score(imdb['sentiment'], gb_predictions, pos_label='positive')

print("\nGradient Boosting Evaluation Metrics:")
print(f"Best Parameters: {best_gb_params}")
print(f"Accuracy: {gb_accuracy}")
print(f"Precision: {gb_precision}")
print(f"Recall: {gb_recall}")
print(f"F1 Score: {gb_f1}")



Gradient Boosting Evaluation Metrics:
Accuracy: 0.79588
Precision: 0.7420009160505137
Recall: 0.9072
F1 Score: 0.8163265306122449
